# Price forecasting with LSTM

This notebook tries to forecast price of NEAR/USDT using the data from data/near_mod.csv. Which is a modified version of the ohlcv data obtained from binance on the 15m timeframe. It adds 3 support and resistance levels to the data and also tracks a midpoint of the candle. The objective  of this code is to use the data to forecast the midpoint of the next candle. The expectation is that, the open price should either be above or below the midpoint of the candle. If we're able to get a 70% accuracy, then we can use this to trade.

If using codespaces, you might want to use to get tensorflow with cuda support
`python3 -m pip install tensorflow[and-cuda]!`

In [4]:
!pip install tensorflow pandas numpy

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import LSTM, Dense

In [9]:
candle = pd.read_csv('../data/near_mod.csv')

In [10]:
candle

,Unnamed: 0,timestamp,open,high,low,close,volume,resistance_1,support_1,resistance_2,support_2,resistance_3,support_3,price_midpoint_hl,price_midpoint_oc,price_midpoint_hlc,price_midpoint_ohlc,midpoint_deviation
0,0,1602651600000,1.3000,1.8800,1.1111,1.3670,5673429.71,1.301329,1.292921,1.304132,1.287315,1.308336,1.284513,1.49555,1.33350,1.452700,1.414525,29.069149
1,1,1602652500000,1.3670,1.3735,1.2222,1.3071,2070780.33,1.367195,1.361590,1.375604,1.354583,1.378406,1.351780,1.29785,1.33705,1.300933,1.317450,2.653804
2,2,1602653400000,1.3070,1.3900,1.3048,1.3689,1048587.47,1.308336,1.304132,1.311139,1.301329,1.315343,1.292921,1.34740,1.33795,1.354567,1.342675,3.744604
3,3,1602654300000,1.3687,1.3838,1.3200,1.3286,352061.02,1.375604,1.367195,1.378406,1.361590,1.382611,1.354583,1.35190,1.34865,1.344133,1.350275,2.540107
4,4,1602655200000,1.3292,1.3385,1.1500,1.2509,1511174.34,1.330759,1.327956,1.333562,1.320949,1.336364,1.318146,1.24425,1.29005,1.246467,1.267150,3.619724
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113625,113625,1704998700000,3.6700,3.6790,3.6330,3.6460,117164.10,3.670000,2.367796,3.670000,2.349578,3.670000,2.345373,3.65600,3.65800,3.652667,3.657000,0.570807
113626,113626,1704999600000,3.6460,3.6540,3.6210,3.6280,86361.50,3.646000,2.367796,3.646000,2.349578,3.646000,2.345373,3.63750,3.63700,3.634333,3.637250,0.465244
113627,113627,1705000500000,3.6290,3.6490,3.6210,3.6310,104546.00,3.629000,2.367796,3.629000,2.349578,3.629000,2.345373,3.63500,3.63000,3.633667,3.632500,0.520691
113628,113628,1705001400000,3.6310,3.6610,3.6290,3.6520,106148.80,3.631000,2.367796,3.631000,2.349578,3.631000,2.345373,3.64500,3.64150,3.647333,3.643250,0.532641
